# Proyecto Final
## KAN para Clasificación de Reacciones Químicas

### Configuración del Entorno

Se importan las librerías necesarias:
- `kan` para implementar Kolmogorov-Arnold Networks.
- `torch` para trabajar con tensores y GPU.
- `numpy` y `pandas` para análisis de datos.

Se configura el dispositivo (`cuda` o `cpu`) y se establece la precisión de los tensores en `float64` para cálculos más precisos. Finalmente, se verifica el dispositivo activo.


In [1]:
from kan import *
import torch
import numpy as np
import pandas as pd
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Carga y preprocesamiento del dataset

1. **Carga del Dataset:**
   - Se carga el archivo `schneider50k.tsv` usando `pandas`, separando las columnas por tabulaciones (`sep='\t'`).

2. **Cálculo de Superclase:**
   - Se agrega una columna llamada `superclass` que contiene el primer dígito de la columna `rxn_class`, ajustado para que las clases comiencen desde `0`.

In [ ]:
dataset = pd.read_csv('/home/jesus_gh/MetodosBiomAI/KAN-RXNFP/schneider50k.tsv',sep='\t')
dataset['superclass'] = dataset['rxn_class'].apply(lambda x: int(str(x).split('.')[0]) - 1)

3. **Eliminación de Columnas Innecesarias:**
   - Se eliminan las columnas `source` y `split` para limpiar el dataset.

4. **Visualización:**
   - Se muestran las primeras filas del dataset modificado para verificar los cambios.

In [3]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['source', 'split',]  

# Eliminar las columnas del dataset original
dataset.drop(columns=columnas_a_eliminar, inplace=True)

# Mostrar las primeras filas del dataset modificado
dataset.head()

,Unnamed: 0,original_rxn,rxn_class,rxn,superclass
0,0,[CH3:17][S:14](=[O:15])(=[O:16])[N:11]1[CH2:10...,6.1.5,C1CCCCC1.CCO.CS(=O)(=O)N1CCN(Cc2ccccc2)CC1.[OH...,5
1,1,O.O.[Na+].[CH3:1][c:2]1[cH:7][c:6]([N+:8](=O)[...,7.1.1,CCOC(C)=O.Cc1cc([N+](=O)[O-])ccc1NC(=O)c1ccccc...,6
2,2,[CH3:1][O:2][c:3]1[cH:4][cH:5][c:6](-[c:9]2[cH...,1.8.5,COc1ccc(-c2coc3ccc(-c4nnc(S)o4)cc23)cc1.COc1cc...,0
3,3,Cl.[CH3:43][CH2:42][S:44](=[O:45])(=[O:46])Cl....,2.2.3,CCS(=O)(=O)Cl.CN(C(=O)N(C)[C@@H]1CN(C(=O)C2CCN...,1
4,4,[CH3:25][O:24][c:21]1[cH:22][cH:23][c:17]([O:1...,1.3.7,COc1ccc(OC)c(N)c1.Cc1cc(Cl)nc(-c2ccccn2)n1>>CO...,0


### Reducción del Dataset

Se toma una muestra aleatoria de 150 filas del dataset original usando `dataset.sample(150)`.

- **Motivo:** Debido al tamaño del dataset completo (50,000 filas) y las limitaciones computacionales del entorno, trabajar con un subconjunto más pequeño ayuda a evitar problemas como el colapso del kernel y reduce el tiempo de procesamiento durante el entrenamiento del modelo.


In [4]:
dataset = dataset.sample(150)  # Usar un subconjunto pequeño para probar

### Definición de Entradas y Salidas

- **Entradas (`inputs`)**: Se seleccionan las cadenas SMILES de las reacciones químicas en la columna `rxn`.
- **Salidas (`outputs`)**: Se asignan las etiquetas de las superclases desde la columna `superclass`.

Esto define las características (`inputs`) y las etiquetas de clasificación (`outputs`) para el modelo.


In [5]:
inputs = dataset['rxn'] #Indico cuales son las entradas
outputs  = dataset['superclass'] #Indico cuales son las salidas

### Embedding y Padding de las SMILES

En esta celda, se transforman las cadenas SMILES en un formato numérico adecuado para el modelo mediante **embedding** y **padding**.

#### **1. Embedding (Codificación One-Hot)**
- **¿Qué hace?**:
  Convierte cada carácter de las SMILES en un vector de una matriz **one-hot**, donde cada fila corresponde a un carácter único.
- **Proceso**:
  - **Vocabulario (`smiles_vocab`)**: Lista de caracteres únicos encontrados en las SMILES.
  - **Función `one_hot_encode_smiles`**:
    - Parámetros:
      - `smiles`: Cadena SMILES que será codificada.
      - `vocab`: Vocabulario de caracteres únicos.
      - `max_len`: Longitud máxima permitida para las cadenas.
    - Se inicializa una matriz de ceros con tamaño `[max_len, vocab_size]`.
    - Cada carácter se asigna a su índice correspondiente en el vocabulario (`char_to_idx`) y se marca con un `1` en la matriz.

#### **2. Padding**
- **¿Qué hace?**:
  Asegura que todas las SMILES tengan la misma longitud (`max_length`), añadiendo ceros para las cadenas más cortas o truncando las más largas.
- **Motivo**:
  Los modelos KAN requieren entradas con dimensiones uniformes, lo que evita errores durante el procesamiento.

#### **3. Implementación con Pandas y PyTorch**
- **Librerías utilizadas**:
  - `pandas`: Para manipular las columnas del dataset y aplicar la codificación a cada SMILES con `.apply()`.
  - `numpy`: Para inicializar y manejar las matrices one-hot.
  - `torch`: Para convertir las listas codificadas en tensores adecuados para el modelo (`torch.tensor`).
- **Finalización**:
  - Las SMILES codificadas y rellenadas se almacenan en `inputs_tokenized`.
  - Se convierten a tensores con `dtype=torch.float64` para mantener precisión y compatibilidad con el modelo.


In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Crear una lista de caracteres únicos encontrados en SMILES
smiles_vocab = list(set("".join(dataset['rxn'])))
max_length = dataset['rxn'].str.len().max()  # Longitud máxima de SMILES

# Tokenización con padding
def one_hot_encode_smiles(smiles, vocab, max_len):
    vocab_size = len(vocab)
    char_to_idx = {char: idx for idx, char in enumerate(vocab)}
    encoding = np.zeros((max_len, vocab_size))  # Matriz con padding
    for i, char in enumerate(smiles[:max_len]):  # Truncar si es necesario
        encoding[i, char_to_idx[char]] = 1
    return encoding.flatten()

# Aplicar la codificación one-hot con padding a todas las entradas SMILES
inputs_tokenized = dataset['rxn'].apply(lambda x: one_hot_encode_smiles(x, smiles_vocab, max_length)).tolist()

# Convertir las listas tokenizadas en tensores
inputs_tensor = torch.tensor(inputs_tokenized, dtype=torch.float64, device=device)


/tmp/ipykernel_446637/3186771519.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  inputs_tensor = torch.tensor(inputs_tokenized, dtype=torch.float64, device=device)


### División del Dataset en Conjuntos de Entrenamiento y Prueba

En esta celda, se separan las entradas y salidas en conjuntos de entrenamiento y prueba.

#### **1. Conversión de Salidas a Tensores:**
- **Propósito:** Las KANs requieren que las etiquetas (`outputs`) estén en formato tensorial para ser procesadas.
- **Implementación:**
  - Se utiliza `torch.tensor` para convertir las etiquetas `outputs` en tensores de tipo `float64` y compatibilidad con el dispositivo (`device`).

#### **2. División del Dataset:**
- **Librería:** `train_test_split` de `scikit-learn`.
- **Propósito:** Divide los datos en dos partes:
  - `train_inputs` y `train_outputs` (70%): Para entrenar el modelo.
  - `test_inputs` y `test_outputs` (30%): Para evaluar el modelo.
- **Parámetros Importantes:**
  - `test_size=0.3`: El 30% de los datos se asigna al conjunto de prueba.
  - `random_state=42`: Semilla para asegurar reproducibilidad.

#### **3. Verificación de Dimensiones:**
- **Por qué es importante:** Permite confirmar que los tamaños de las particiones son correctos y consistentes con el formato esperado por el modelo.
- **Resultado:** Se imprimen las dimensiones de los conjuntos de entrenamiento y prueba.

Esto garantiza que los datos estén listos para ser utilizados por el modelo con una separación clara entre entrenamiento y prueba.

In [7]:
from sklearn.model_selection import train_test_split
import torch

# Convertir etiquetas de superclases en tensores
outputs_tensor = torch.tensor(outputs.values, dtype=torch.float64, device=device)

# Dividir en conjuntos de entrenamiento y prueba
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs_tensor, outputs_tensor, test_size=0.3, random_state=42)

# Imprimo las dimensiones de los conjuntos de entrenamiento y prueba
print(f'Train set: {train_inputs.shape}, {train_outputs.shape}') 
print(f'Test set: {test_inputs.shape}, {test_outputs.shape}')


Train set: torch.Size([105, 13120]), torch.Size([105])
Test set: torch.Size([45, 13120]), torch.Size([45])


In [8]:
# Imprimir los tipos de datos de los tensores
print(f"Train input dtype: {train_inputs.dtype}")
print(f"Test input dtype: {test_inputs.dtype}")

Train input dtype: torch.float64
Test input dtype: torch.float64


### Creación del Diccionario de Dataset

Se crea un diccionario llamado `dataset_kan` que contiene los conjuntos de datos de entrada y salida para entrenamiento y prueba. Este formato facilita el acceso y manejo de los datos en el modelo KAN, agrupando las entradas y salidas de manera estructurada.

Este paso es crucial para organizar los datos de forma eficiente antes de ser utilizados por el modelo.

In [9]:
# Hago un diccionario con los conjuntos de datos
dataset_kan = {
    'train_input': train_inputs,
    'train_label': train_outputs,
    'test_input': test_inputs,
    'test_label': test_outputs,
}

### Declaración del Modelo KAN

En esta celda, se declara el modelo **KAN** con la siguiente estructura:
`"model = KAN(width=[train_inputs.shape[1], [5, 2], 1], grid=3, k=3)`

#### Explicación del Modelo:

*width=[train_inputs.shape[1], [5, 2], 1]:*

* `train_inputs.shape[1]`: El número de características en los datos de entrada.
* `[5, 2]`: La capa oculta tiene 5 neuronas de adición y 2 neuronas de multiplicación (esto corresponde a un MultKAN).
* Multiplicación en MultKAN: La multiplicación explícita en las capas permite que el modelo capture interacciones no lineales entre las características, lo que mejora su capacidad para aprender patrones más complejos en los datos.
* 1: Una salida del modelo.
* `grid=3, k=3`: Configuración de la complejidad y el número de intervalos dentro de la red.

El uso de multiplicación en las capas ocultas mejora la capacidad del modelo para modelar relaciones no lineales y complejas entre las entradas y las superclases, a diferencia de las redes tradicionales que solo utilizan sumas.

In [ ]:
# Inicializo el modelo KAN
model = KAN(width=[train_inputs.shape[1], [5,2], 1], grid=3, k=3)

checkpoint directory created: ./model
saving model version 0.0


In [ ]:
"""Definir una métrica de precisión para entrenamiento y prueba
def train_acc():
    return torch.mean((torch.round(model(train_inputs)) == train_outputs).float()) # Aproximación de la precisión
# round redondea a 0 o 1


def test_acc():
    return torch.mean((torch.round(model(test_inputs)) == test_outputs).float())

Luego probar con argmax en vez de torch"""

### Definición de Métricas Personalizadas (Precisión de Entrenamiento y Prueba)

En esta celda, se definen las funciones de precisión personalizadas para evaluar el desempeño del modelo durante el entrenamiento y la prueba. Se utiliza `torch.argmax()` en lugar de `round()` para obtener la clase predicha, lo que mejora la precisión.

- **`torch.argmax()`**: Esta función devuelve el índice del valor máximo en un tensor a lo largo de un eje. En el caso de la clasificación, esto corresponde a la clase predicha, ya que el valor máximo de las salidas del modelo representa la clase más probable.
  - **¿Por qué `argmax` es mejor que `round()`?**
    - `round()` es más adecuado para predicciones continuas (como regresión), mientras que `argmax()` es específico para clasificación, ya que selecciona la clase con la mayor probabilidad.
  
- **Función `train_acc()` y `test_acc()`**:
  - Calculan la precisión comparando las clases predichas con las clases reales.
  - **Cálculo:** Compara la salida de `argmax()` con la etiqueta real (`train_outputs` o `test_outputs`) y calcula el porcentaje de coincidencias.

Este enfoque garantiza una evaluación adecuada para problemas de clasificación.

In [11]:
# Definir las funciones de métrica personalizadas
# argmax nos da el indice del valor maximo de la fila, lo que nos da la clase
def train_acc():
    predictions = model(train_inputs)
    return torch.mean((torch.argmax(predictions, dim=1) == train_outputs).float()) 

def test_acc():
    predictions = model(test_inputs)
    return torch.mean((torch.argmax(predictions, dim=1) == test_outputs).float())

### Ajuste de la Forma de las Etiquetas

La función `train_outputs.view(-1, 1)` asegura que las etiquetas de entrenamiento tengan la forma correcta, convirtiéndolas en un tensor de una sola columna (forma `(N, 1)`), donde `N` es el número de ejemplos. Esto es necesario para que las etiquetas sean compatibles con las salidas del modelo y la función de pérdida, especialmente en problemas de clasificación donde cada etiqueta debe estar en una dimensión específica.


In [12]:
train_outputs = train_outputs.view(-1, 1) # Asegurarse de que las etiquetas tengan la forma correcta

### Entrenamiento del Modelo y Evaluación de Precisión

1. **Entrenamiento del Modelo**:
   - `model.fit(dataset_kan, opt="LBFGS", steps=50, metrics=(train_acc, test_acc))`: El optimizador **LBFGS** (Limited-memory Broyden-Fletcher-Goldfarb-Shanno) es un método de optimización que busca minimizar la función de pérdida ajustando los parámetros del modelo. Es especialmente útil en modelos que involucran funciones complejas, como KANs, y es eficiente en problemas de optimización no lineales y de alta dimensionalidad.
   - **Uso de LBFGS**: Es adecuado cuando se busca precisión en modelos donde los gradientes y la optimización son complicados, como redes neuronales profundas.

2. **Impresión de la Precisión**:
   - `results['train_acc'][-1], results['test_acc'][-1]`: Muestra la precisión final del modelo tanto en el conjunto de entrenamiento como en el de prueba.

El optimizador **LBFGS** ayuda a mejorar la convergencia del modelo en escenarios complejos y reduce el riesgo de estancamiento en óptimos locales.

In [ ]:
results = model.fit(dataset_kan, opt="LBFGS",steps=50, metrics=(train_acc, test_acc)) # Entrenar el modelo
results['train_acc'][-1], results['test_acc'][-1] # Imprimir la precisión del modelo

description:   0%|                                                           | 0/50 [00:00<?, ?it/s]/home/jesus_gh/miniconda3/envs/KAN_rxnfp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
| train_loss: 3.27e+00 | test_loss: 3.30e+00 | reg: 1.78e+02 | : 100%|█| 50/50 [29:47<00:00, 35.75s/


saving model version 0.1


(0.2857142984867096, 0.3333333432674408)

### Impresión de la Precisión Final

En esta celda, se imprime la precisión final del modelo tanto para el conjunto de entrenamiento como para el conjunto de prueba:

- **`results['train_acc'][-1]`**: Muestra la precisión obtenida en el conjunto de entrenamiento al final del proceso de entrenamiento.
- **`results['test_acc'][-1]`**: Muestra la precisión obtenida en el conjunto de prueba, lo que ayuda a evaluar la capacidad de generalización del modelo.

Esto proporciona una medida clave para evaluar el rendimiento del modelo después del entrenamiento.


In [14]:
print(f"Final train accuracy: {results['train_acc'][-1]}")
print(f"Final test accuracy: {results['test_acc'][-1]}")

Final train accuracy: 0.2857142984867096
Final test accuracy: 0.3333333432674408


### Generación de la Gráfica del Modelo

El comando `model.plot()` se utiliza para generar una visualización gráfica del modelo KAN entrenado. Esto permite ver cómo el modelo ha aprendido y cómo están distribuidas las funciones de activación a lo largo de las capas. 

**Nota:** La generación de la gráfica puede ser computacionalmente costosa, especialmente con grandes modelos o datasets, por lo que puede causar problemas de rendimiento o incluso hacer que el kernel se detenga. A pesar de ello, este paso es útil para interpretar y visualizar el comportamiento del modelo.


In [ ]:
model.plot()


### Activación Simbólica Automática

Se utiliza `model.auto_symbolic(lib=lib)` para activar las funciones simbólicas en el modelo KAN. Esto convierte las funciones de activación en representaciones más comprensibles y matemáticamente interpretables, utilizando una librería de funciones predefinidas (`lib`).

- **`lib`**: Lista de funciones simbólicas (como `x`, `x^2`, `exp`, etc.) que el modelo utilizará para expresar sus activaciones.
- **¿Por qué es útil?**: Esto permite obtener una comprensión más profunda de cómo el modelo genera sus predicciones y qué funciones matemáticas está aprendiendo.

Este paso facilita la interpretación del modelo y su comportamiento en términos de funciones conocidas.


In [ ]:
lib = ['x','x^2']

model.auto_symbolic(lib=lib)

### Obtención de la Fórmula Simbólica del Modelo

En esta celda, se extrae la fórmula simbólica del modelo entrenado utilizando el método `model.symbolic_formula()`. Este método devuelve las representaciones simbólicas de las funciones que el modelo ha aprendido.

- **`model.symbolic_formula()[0]`**: Extrae la primera fórmula simbólica generada por el modelo.
- **`ex_round(formula1, 4)`**: La función `ex_round` se utiliza para redondear la fórmula a 4 decimales, facilitando su interpretación y presentación.

Estas fórmulas proporcionan una forma matemática de comprender cómo el modelo toma decisiones y puede ser útil para visualizar los patrones aprendidos por el KAN.


In [ ]:
from kan.utils import ex_round

ex_round(model.symbolic_formula()[0][0],4)

### Poda del Modelo

El comando `model = model.prune()` se utiliza para **podar** el modelo, es decir, eliminar las conexiones o parámetros innecesarios dentro de la red para simplificar el modelo sin perder capacidad de aprendizaje. 

**¿Por qué usar poda?**
- La poda ayuda a mejorar la eficiencia del modelo, reduciendo la complejidad computacional y acelerando la inferencia sin sacrificar significativamente el rendimiento.

Este paso es útil para optimizar el modelo una vez que ha sido entrenado y ajustado.


In [ ]:
model = model.prune()

saving model version 0.2


### Visualización del Modelo Podado

El comando `model.plot()` se utiliza para visualizar el modelo **pruneado**, lo que muestra la estructura del modelo después de haber eliminado los parámetros innecesarios. Esta visualización ayuda a ver cómo el modelo ha sido simplificado y optimizado, permitiendo una interpretación más clara de la red y sus conexiones.

**Nota**: La visualización puede ser computacionalmente costosa, especialmente con modelos grandes, y podría requerir recursos adicionales.


In [19]:
model.plot()

: 